https://pythonprogramming.net/sentiment-analysis-module-nltk-tutorial/

### Building the module for sentiment analysis

this one builds on the previous Notebooks

Here everything is loaded into the file

In [1]:
import nltk
import random
import pickle
from nltk.classify.scikitlearn import SklearnClassifier
import pickle
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.svm import SVC, LinearSVC, NuSVC
from nltk.classify import ClassifierI
from statistics import mode
from nltk.tokenize import word_tokenize


In [2]:
class VoteClassifier(ClassifierI):
    # init method will always run in a class
    def __init__(self, *classifiers):
        self._classifiers = classifiers
        
    # classify method
    def classify(self, features):
        votes = []
        for c in self._classifiers:
            v = c.classify(features)
            votes.append(v)
        return mode(votes)
    
    def confidence(self, features):
        votes = []
        for c in self._classifiers:
            v = c.classify(features)
            votes.append(v)
        choice_votes = votes.count(mode(votes))
        conf = choice_votes / len(votes)
        return conf
    

In [3]:
documents_f = open("pickled_algorithms/documents.pickle", "rb")
documents = pickle.load(documents_f)
documents_f.close()


In [4]:
word_features_f = open("pickled_algorithms/word_features.pickle", "rb")
word_features = pickle.load(word_features_f)
word_features_f.close()


In [5]:
def find_features(document):
    words = word_tokenize(document)
    features = {}
    for w in word_features:
        features[w] = (w in words)

    return features


In [6]:
featuresets_f = open("pickled_algorithms/featuresets.pickle", "rb")
featuresets = pickle.load(featuresets_f)
featuresets_f.close()

random.shuffle(featuresets)
print(len(featuresets))


10662


In [7]:
# Train and test sets
training_set = featuresets[:10000]
testing_set = featuresets[10000:]


In [8]:
# Load the pickled classifiers
open_file = open("pickled_algorithms/naivebayes.pickle", "rb")
classifier = pickle.load(open_file)
open_file.close()

open_file = open("pickled_algorithms/MNB_classifier.pickle", "rb")
MNB_classifier = pickle.load(open_file)
open_file.close()

open_file = open("pickled_algorithms/BernoulliNB_classifier.pickle", "rb")
BernoulliNB_classifier = pickle.load(open_file)
open_file.close()

open_file = open("pickled_algorithms/LogisticRegression_classifier.pickle", "rb")
LogisticRegression_classifier = pickle.load(open_file)
open_file.close()

open_file = open("pickled_algorithms/LinearSVC_classifier.pickle", "rb")
LinearSVC_classifier = pickle.load(open_file)
open_file.close()

open_file = open("pickled_algorithms/SGD_classifier.pickle", "rb")
SGDC_classifier = pickle.load(open_file)
open_file.close()


In [9]:
voted_classifier = VoteClassifier(classifier,
                                  LinearSVC_classifier,
                                  SGDC_classifier,
                                  MNB_classifier,
                                  BernoulliNB_classifier,
                                  LogisticRegression_classifier)


In [10]:
def sentiment(text):
    feats = find_features(text)
    
    return voted_classifier.classify(feats), voted_classifier.confidence(feats)


#### Test the classifier

In [13]:
REVIEW = input('Enter your movie review: \n')

Enter your movie review: 
This movie was great, the acting was excellent and it deserves all the praise.


In [14]:
print(sentiment(REVIEW))

('pos', 1.0)
